In [4]:
import pandas as pd

file = r"C:\Users\User\Downloads\nasa25\cumulative_2025.09.30_03.59.13.csv"

df = pd.read_csv(file, sep=",", quotechar='"', on_bad_lines="skip", low_memory=False)


print(df.shape)
print(df.head())


(143, 1)
  # This file was produced by the NASA Exoplanet Archive  http://exoplanetarchive.ipac.caltech.edu
0                         # Tue Sep 30 03:59:13 2025                                              
1                                                  #                                              
2                     # COLUMN kepid:          KepID                                              
3                  # COLUMN kepoi_name:     KOI Name                                              
4               # COLUMN kepler_name:    Kepler Name                                              


In [10]:
df.describe()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
count,9564.000000,9564.000000,9.564000e+03,9564.000000,9.564000e+03,9110.000000,9110.000000,9564.000000,9109.000000,9109.000000,...,8965.000000,8965.000000,8965.000000,8965.000000,8993.000000,8993.000000,8993.000000,8993.000000,8993.000000,8993.000000
mean,0.208853,0.232748,1.975115e-01,0.132869,3.323582e+02,0.002148,-0.002148,166.170415,0.009934,-0.009934,...,-0.045416,0.446230,1.866558,0.489931,-0.024248,0.425377,-0.076727,0.436713,1.812737,0.476094
std,4.767346,0.422605,3.981423e-01,1.295508,2.513759e+04,0.008236,0.008236,67.938764,0.023098,0.023098,...,2.573557,0.569932,2.988743,0.646403,2.382418,0.602172,2.553899,0.567560,2.986498,0.647615
min,0.000000,0.000000,-2.700000e-08,0.000000,2.418425e-01,0.000000,-0.172500,0.000000,0.000013,-0.569000,...,-75.900000,0.067000,0.000000,0.067000,-27.800000,0.067000,-76.600000,0.067000,0.000000,0.067000
25%,0.000000,0.000000,0.000000e+00,0.000000,2.734862e+00,0.000005,-0.000276,132.761718,0.001240,-0.010500,...,-0.387000,0.098000,0.170000,0.100000,-0.310000,0.087000,-0.390000,0.090000,0.210000,0.094000
50%,0.000000,0.000000,0.000000e+00,0.000000,9.757456e+00,0.000035,-0.000035,137.224595,0.004130,-0.004130,...,0.000000,0.280000,0.610000,0.310000,-0.004000,0.250000,-0.017000,0.270000,0.585000,0.290000
75%,0.000000,0.000000,0.000000e+00,0.000000,4.071873e+01,0.000276,-0.000005,170.694603,0.010500,-0.001240,...,0.300000,0.610000,2.160000,0.680000,0.290000,0.590000,0.300000,0.600000,1.970000,0.660000
max,465.000000,1.000000,1.000000e+00,122.763305,2.454956e+06,0.172500,0.000000,1472.522306,0.569000,-0.000013,...,27.500000,22.000000,88.600000,32.000000,46.570000,33.000000,34.000000,22.000000,89.600000,32.000000


In [13]:
df.describe(include=[object])

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_stat,koi_vet_date,koi_pdisposition,koi_score,koi_disp_prov,...,koi_parm_prov,koi_max_sngle_ev,koi_count,koi_tce_delivname,koi_quarters,koi_trans_mod,koi_datalink_dvr,koi_datalink_dvs,koi_smass,koi_sparprov
count,9565,9565,9565,2748,9565,9565,9565,9564,8054,9564,...,9564,8422,9564,9218,8422,9201,9218,9218,9201,9201
unique,9565,8215,9565,2748,4,2,3,3,650,2,...,4,8421,8,4,213,2,8063,9218,1477,5
top,1,11968463,K00752.01,Kepler-227 b,FALSE POSITIVE,Done,2018-08-16,FALSE POSITIVE,0.0000,q1_q17_dr25_sup_koi,...,q1_q17_dr25_koi,4.2350397,1,q1_q17_dr25_tce,11111111111111111000000000000000,Mandel and Agol (2002 ApJ 580 171),011/011442/011442793/dv/kplr011442793-20160209...,010/010797/010797460/dv/kplr010797460-001-2016...,1.0000,q1_q17_dr25_stellar
freq,1,7,1,1,4839,9564,9563,4847,3482,9563,...,8053,2,7275,8053,5426,9200,7,1,124,7906


In [ ]:
df.shape

(9565, 141)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9565 entries, 0 to 9564
Columns: 141 entries, rowid to koi_dikco_msky_err
dtypes: float64(115), object(26)
memory usage: 10.3+ MB


In [7]:
import pandas as pd

file = r"C:\Users\User\Downloads\nasa25\cumulative_2025.09.30_03.59.13.csv"

df = pd.read_csv(
    file,
    sep=",",
    quotechar='"',
    skiprows=144,   # skips first 44 lines, so reading begins at line 45
    low_memory=False
)

df.to_csv('nasa_Kepler Objects of Interest (KOI)')


In [9]:
df.isnull().sum()

rowid                    0
kepid                    0
kepoi_name               0
kepler_name           6817
koi_disposition          0
                      ... 
koi_dikco_mra_err      572
koi_dikco_mdec         572
koi_dikco_mdec_err     572
koi_dikco_msky         572
koi_dikco_msky_err     572
Length: 141, dtype: int64

In [ ]:
col